#Select VASCA sources based on chosen criteria

In [2]:
region_name = "ALL_10-800" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_cat_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

### Prepare tt_sources with variables for selection

In [4]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from vasca.region import Region
import vasca.visualization as vvis
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, get_col_cycler,add_ogrp, color_palette, nb_fig
from astropy import units as uu
from astropy.table import Column

# Get region
rc = Region()
rc.load_from_fits(region_cat_fname)
rc.tt_sources.sort(["dec"])

#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp","match_distance","sp_type","main_id"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_simbad")
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
sel_simbad = rc.tt_sources["sel"].data
print(f"Number of SIMBAD matches is: {sel_simbad.sum()}  ({100* sel_simbad.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiadr3",["PQSO", "PGal", "PSS","match_distance","Gmag_abs","BP-RP", "ogrp", "Plx","e_Plx","Plx_dist","VarFlag"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_gaiadr3")
rc.tt_sources.rename_column("ogrp","ogrp_gaiadr3")
sel_gaiadr3 = rc.tt_sources["sel"].data
print(f"Number of GAIA matches is: {sel_gaiadr3.sum()}  ({100*sel_gaiadr3.sum() / len(rc.tt_sources)} )%")

sel_match = np.array(sel_simbad+sel_gaiadr3, dtype=bool)
print(f"Total number of matches is: {sel_match.sum()}  ({100*sel_match.sum() / len(rc.tt_sources)} )%")

#Copy lomb scargle result into tt_sources
rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

display(rc.tt_simbad[rc.tt_simbad["ogrp"]=="WD*"])

Number of SIMBAD matches is: 4761  (56.84097421203438 )%
Number of GAIA matches is: 6216  (74.21203438395415 )%
Total number of matches is: 6885  (82.19914040114614 )%


main_id,ra,dec,ra_prec,dec_prec,coo_err_maja,coo_err_mina,coo_err_angle,coo_qual,coo_wavelength,coo_bibcode,otype,otypes,distance_distance,distance_q,distance_unit,distance_merr,distance_perr,distance_method,distance_bibcode,match_distance,rvz_type,rvz_radvel,rvz_error,rvz_qual,rvz_wavelength,rvz_bibcode,z_value,sp_type,sp_qual,sp_bibcode,script_number_id,rg_src_id,simbad_match_id,ogrp
,deg,deg,,,mas,mas,deg,,,,,,,,,,,,,arcsec,,km / s,km / s,,,,,,,,,,,
bytes32,float64,float64,int16,int16,float32,float32,int16,bytes1,bytes1,bytes32,bytes32,bytes32,float64,bytes1,bytes4,float64,float64,bytes8,bytes19,float32,bytes1,float64,float32,bytes1,bytes1,bytes32,float64,bytes32,bytes32,bytes32,int32,int32,int32,bytes8
V* V2276 Sgr,306.59244125,-43.6754825,14,14,0.331,0.248,90,A,O,2020yCat.1350....0G,CV*,CV*|V*|*|WD?,--,--,--,--,--,--,--,0.2714,--,--,--,--,--,--,--,--,--,--,348,21770,267,WD*
SBSS 1612+554,243.31930499999993,55.357211944444444,14,14,0.031,0.028,90,A,O,2020yCat.1350....0G,WD*,WD*|Pu*|MIR|NIR|WD*|*|Opt|WD?|WD,65.9420,--,pc,-0.1309,0.1309,paral,2020yCat.1350....0G,0.1378,--,--,--,--,--,--,--,DA4.2,C,2011ApJ...743..138G,395,25273,288,WD*
GALEX J161003.8+561350,242.51598291666667,56.23088694444445,14,14,0.128,0.121,90,A,O,2020yCat.1350....0G,WD?,*|UV|WD?|WD?,--,--,--,--,--,--,--,0.3921,--,--,--,--,--,--,--,--,--,--,401,26195,295,WD*
V* IY UMa,160.98645291666665,58.12553333333334,14,14,0.036,0.046,90,A,O,2020yCat.1350....0G,CV*,PM*|CV*|CV*|V*|PM*|NIR|*|*|CV?|C,182.0470,--,pc,-2.2337,2.2337,paral,2020yCat.1350....0G,0.3422,z,0.001,0.000,E,--,2009ApJS..182..543A,0.0005300,--,--,--,494,32448,350,WD*
2QZ J100132.7+013057,150.38664125,1.5162241666666667,14,14,0.364,0.397,90,A,O,2020yCat.1350....0G,WD*,WD*|WD*|blu|*|*|G|Opt|UV|WD?,1110.4600,--,pc,-129.4600,129.4600,ST-L,2017MNRAS.469.2102A,0.5794,v,-74.400,19.330,C,O,2017MNRAS.469.2102A,-0.0002480,DA,D,2015MNRAS.446.4078K,541,34658,390,WD*
M31N 2008-05d,11.007374999999998,41.07325,6,6,--,--,0,C,O,2008ATel.1602....1H,No*,No*,--,--,--,--,--,--,--,0.5632,--,--,--,--,--,--,--,--,--,--,858,53048,581,WD*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V* MQ Dra,238.37966583333332,55.2706875,14,14,0.055,0.066,90,A,O,2020yCat.1350....0G,CV*,EB*|V*|CV*|V*|CV*|NIR|*|Opt|X,186.2340,--,pc,-3.2116,3.2116,paral,2018yCat.1345....0G,0.2065,z,-0.000,0.000,E,--,2008ApJS..175..297A,-0.0003500,M5Ve,D,2003ApJ...583..902S,6442,1232731,3758,WD*


### Print info for selected sources

In [5]:
# Make selection
sel_otype = rc.tt_sources["otype"] ==  "RR*"# "No*"#"HS?" #"BS*" #"PM*" # # "SN*" #"EB*"
sel_otypes = (rc.tt_sources["otype"] ==  "WD?")  + (rc.tt_sources["otype"] =="WD*") + (rc.tt_sources["ogrp_gaiadr3"] =="WD*")
sel_flux_nxv = rc.tt_sources["flux_nxv"][:,0]>2
sel_flux_hr = rc.tt_sources["hr"]>2
sel_src_id = rc.tt_sources["rg_src_id"]==15179
sel_period = (rc.tt_sources["ls_peak_pval"]>-0.5) * (rc.tt_sources["ls_peak_pval"]<0.000000573303) * (rc.tt_sources["ls_model_rchiq"]<1.1)# # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
sel_srcs =  sel_otypes #sel_period #sel_otypes  #sel_src_id  #sel_flux_nxv #sel_otype # sel_flux_nxv_fuv # sel_flux_hr#

#Print oout list of source IDs to be used for further use, e.g. in inspect_sources.ipynb
print("\nsrcs_ids = [", end="")
for src_id in rc.tt_sources[sel_srcs]["rg_src_id"].data:
    print(src_id,",", end="")
print("]\n")
print("Total Nr. of sources:",sel_srcs.sum(),"\n")

#Display match and simbad table for selected sources
cols_write = ["ra","dec","ogrp_gaiadr3","otype","sp_type","main_id","nr_det","flux","flux_cpval","flux_ne","Plx_dist","ls_peak_pval","ls_model_rchiq","VarFlag","rg_src_id"]
display(rc.tt_sources[sel_srcs][cols_write] )#["sp_type","main_id"]
rc.tt_sources[sel_srcs][cols_write].write("sources.ecsv",overwrite=True,delimiter=',')


srcs_ids = [193067 ,432606 ,535864 ,451644 ,1551422 ,541266 ,581995 ,625693 ,187856 ,8215 ,494782 ,166179 ,172775 ,34658 ,98746 ,1521738 ,2136829 ,297278 ,426363 ,426330 ,151796 ,305192 ,259271 ,388172 ,265150 ,54184 ,472623 ,419001 ,25273 ,26195 ,32448 ,199832 ,]

Total Nr. of sources: 32 



ra,dec,ogrp_gaiadr3,otype,sp_type,main_id,nr_det,flux,flux_cpval,flux_ne,Plx_dist,ls_peak_pval,ls_model_rchiq,VarFlag,rg_src_id
deg,deg,,,,,,1e-06 Jy,,,pc,,,,
float64,float64,bytes8,bytes32,bytes32,bytes32,int32[2],float32[2],float32[2],float32[2],float32,float32,float32,bytes32,int32
303.0110051436914,-48.39410878608832,Star*,WD?,,Gaia DR3 6669015352243852800,18 .. 6,145.71657 .. 53.254395,0.0 .. 4.538933e-19,0.10808045 .. 0.30587497,20120.725,-1.0,-1.0,VARIABLE,193067
79.85087717838213,-48.00660083372567,WD*,none,none,none,19 .. 14,10.857628 .. 4.288318,3.626451e-12 .. 0.092903994,0.17871924 .. 0.15028061,433.78302,-1.0,-1.0,NOT_AVAILABLE,432606
17.908561463231784,-46.124209856672074,WD*,none,none,none,12 .. 2,210.91878 .. 51.06775,4.861283e-20 .. 0.8805627,0.05030113 .. --,152.56229,-1.0,-1.0,VARIABLE,535864
53.603003894660404,-35.35703047345705,WD*,WD?,,GALEX J033424.7-352125,24 .. 28,309.9156 .. 470.0984,0.106752574 .. 9.816341e-09,0.011271525 .. 0.025724309,430.47784,0.5929085,0.87381697,NOT_AVAILABLE,451644
207.03531485024092,-30.479189687581158,Star*,WD?,,Gaia DR3 6175066552397551616,15 .. -1,144.80975 .. -1.0,1.1032699e-20 .. -1.0,0.060247365 .. --,nan,-1.0,-1.0,NOT_AVAILABLE,1551422
12.321346896575124,-25.432354111852234,WD*,WD*,DA,GALEX J004917.1-252556,11 .. 1,73.65471 .. 41.309494,1.0632669e-08 .. -1.0,0.04259176 .. --,99.60556,-1.0,-1.0,NOT_AVAILABLE,541266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49.89487839440356,41.93511662149011,WD*,none,none,none,12 .. 10,146.99852 .. 48.611095,7.10371e-20 .. 0.005771271,0.0510717 .. 0.05662148,204.54918,-1.0,-1.0,NOT_AVAILABLE,472623
